In [1]:
import pandas as pd

from scripts.processing import process_datasets, load_datasets
from scripts.scorer import scorer, simulate_scorer

In [2]:
process_datasets(
    ballots_csv_path='data/_local/rf6_votes.csv',
    voters_csv_path='data/_local/rf6_voters.csv',
    projects_csv_path='data/_local/rf6_projects.csv',
    output_dir='data/_local/clean'
)

voting_data, df_all_votes, df_voters, df_projects = load_datasets('data/_local/clean')

Voters dataframe length: 178
Actual voters length: 138
Voters dataframe length after filtering: 138
Processing complete.


In [3]:
all_voters = citizens = (
    simulate_scorer(
        ballots_data=voting_data,
        addresses=list(df_voters.index)
    ).join(
        df_projects[['name', 'organization', 'category']]
    )
).to_csv('data/_local/rf6_results_final.csv')


Median Budget: 2,400,000

Category: GOVERNANCE_INFRA_AND_TOOLING
-----------------
Median Allocation: 41.891%

Category: GOVERNANCE_LEADERSHIP
-----------------
Median Allocation: 27.961%

Category: GOVERNANCE_ANALYTICS
-----------------
Median Allocation: 30.149%


In [4]:
citizens = (
    simulate_scorer(
        ballots_data=voting_data,
        addresses=list(df_voters[df_voters['is_citizen']==True].index)
    ).join(
        df_projects[['name', 'organization', 'category']]
    )
).to_csv('data/_local/rf6_results_guests.csv')


Median Budget: 2,000,000

Category: GOVERNANCE_INFRA_AND_TOOLING
-----------------
Median Allocation: 50.000%

Category: GOVERNANCE_LEADERSHIP
-----------------
Median Allocation: 25.000%

Category: GOVERNANCE_ANALYTICS
-----------------
Median Allocation: 25.000%


In [5]:
guests = (
    simulate_scorer(
        ballots_data=voting_data,
        addresses=list(df_voters[df_voters['is_citizen']==False].index)
    ).join(
        df_projects[['name', 'organization', 'category']]
    )
).to_csv('data/_local/rf6_results_guests.csv')


Median Budget: 2,500,000

Category: GOVERNANCE_INFRA_AND_TOOLING
-----------------
Median Allocation: 40.070%

Category: GOVERNANCE_LEADERSHIP
-----------------
Median Allocation: 29.840%

Category: GOVERNANCE_ANALYTICS
-----------------
Median Allocation: 30.090%
